<p style="text-align: justify;"> Justified text cell. </p>

# <p style="text-align: center;"> Variational Quantum Linear Solver on Rigetti QCS </p>

<p style="text-align: center;"> Ryan LaRose, Department of Computational Mathematics, Science, and Engineering, Michigan State University </p>

<img src="main.png" alt="Overview of VQLS">

### <p style="text-align: center;"> Abstract </p>

<p style="text-align: justify;"> The variational quantum linear solver (VQLS) [1] is an algorithm for solving linear systems on near-term quantum computers. While VQLS does not offer performance guaruntees like other quantum linear systems algorithms [2-3], the ability to run on near-term computers makes it an interesting problem-specific benchmark. In this notebook, we provide a brief tutorial of VQLS and show results of running VQLS on Rigetti QCS for two example linear systems. </p>

## <p style="text-align: center;"> Notebook Setup </p>

<p style="text-align: justify;"> We use pyQuil to write the VQLS algorithm and send job submissions to Rigetti QCS. In the notebook, we use the Rigetti QVM for tutorial purposes. When running on the device, the code written in the notebook is converted to a script and executed on the QPU. </p>

In [ ]:
"""Imports."""
from itertools import product
from math import pi
import time
from typing import List, Tuple

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from scipy.optimize import minimize

import pyquil
from pyquil import Program, get_qc
import pyquil.gates as gates

<p style="text-align: justify;"> Here we define a few helper functions for the notebook. </p>

In [ ]:
"""Helper functions and useful constants."""
imat = np.array([[1, 0], [0, 1]], dtype=np.complex64)
xmat = np.array([[0, 1], [1, 0]], dtype=np.complex64)
ymat = np.array([[0, -1j], [1j, 0]], dtype=np.complex64)
zmat = np.array([[1, 0], [0, -1]], dtype=np.complex64)
pauli_dict = {"I": imat, "X": xmat, "Y": ymat, "Z": zmat}


def tensor(*matrices) -> np.ndarray:
    """Returns the tensor product of all matrices."""
    mats = list(matrices)
    mat = mats[0]
    if type(mat) != np.ndarray:
        raise ValueError("All arguments must be numpy.ndarray's.")
    for term in mats[1:]:
        mat = np.kron(mat, term)
    return mat


def matrix(coeffs: List[complex], terms: List[str]):
    """Returns the matrix defined by the coefficients and Paulis.
    
    Args:
        coeffs: List of real/complex numbers specifying coefficients of Paulis.
        terms: List of Pauli operators.
    """
    if len(coeffs) != len(terms):
        raise ValueError("Number of coeffs does not equal number of terms.")
    nqubits = len(terms[0])
    dim = 2**nqubits
    mat = np.zeros((dim, dim), dtype=np.complex64)
    for (c, pauli) in zip(coeffs, terms):
        pauli = [pauli_dict[key] for key in pauli]
        mat += c * tensor(*pauli)
    return mat


def vector(coeffs: List[complex], terms: List[str]):
    """Returns the vector of the linear system.
    
    Args:
        coeffs: Weights of B matrix.
        terms: Pauli terms in B matrix.
    """
    return matrix(coeffs, terms)[:, 0]

In [ ]:
"""Unit tests."""
assert np.array_equal(matrix([1], ["Y"]), ymat)
assert np.array_equal(vector([1], ["Y"]), ymat[:, 0])

## <p style="text-align: center;"> Example Linear Systems </p>

A linear system $A \mathbf{x} = \mathbf{b}$ is defined by a matrix $A \in \mathbb{C}^{N \times N}$ and vector $\mathbf{b} \in \mathbb{C}^{N}$. Given $A$ and $\mathbf{b}$ as input, the goal is to output $\mathbf{x} \in \mathbb{C}^{N}$, or an approximate solution $\mathbf{\bar{x}}$ where $|| \mathbf{\bar{x}} - \mathbf{x} || \le \delta$. 

### <p style="text-align: center;"> Linear System 1 </p>

<p style="text-align: justify;"> The first linear system we consider is an example from the VQLS paper. Our motivation for this is to compare results from the paper (which used older Rigetti computers) to results we get on the new computers. The linear system is a five-qubit example given by </p>

\begin{equation}
    A_1 = I + 0.2 X_1 + 0.2 X_1 Z_2 .
\end{equation}

and $\mathbf{b}_1 = H_1 H_2 H_4 H_4 |0\rangle^{\otimes 5}$. Here, $I, X, Y$ and $Z$ are the usual Pauli operators

\begin{equation}
    I := \left[ \begin{matrix}
    1 & 0 \\
    0 & 1 \\
    \end{matrix} \right], \qquad 
    X := \left[ \begin{matrix}
    0 & 1 \\
    1 & 0 \\
    \end{matrix} \right], \qquad 
    Y := \left[ \begin{matrix}
    0 & -i \\
    i & 0 \\
    \end{matrix} \right], \qquad 
    Z := \left[ \begin{matrix}
    1 & 0 \\
    0 & -1 \\
    \end{matrix} \right]
\end{equation}

and subscripts indicate which qubits the Paulis act on. For example, $X_1$ means $X$ on the first qubit, i.e. $X I I I I$. Using the helper functions above, we can view the $32 \times 32$ matrix of this linear system.

In [ ]:
"""View the matrix of the linear system."""
Acoeffs = [1., 0.2, 0.2]
Aterms = ["IIIII", "XIIII", "XZIII"]
print("The matrix of the linear system is:")
print(matrix(Acoeffs, Aterms))

### <p style="text-align: center;"> Linear System 2 </p>

The second linear system we consider is formed from the Ising model.

\begin{equation}
    A_2 := \frac{1}{\zeta} \left(\sum_{j=1}^{n} X_j + J \sum_{j=1}^{n-1}Z_jZ_{j+1} + \eta I \right)
\end{equation}

# <p style="text-align: center;"> The VQLS Algorithm </p>

### <p style="text-align: center;"> Problem Setup </p>

<p style="text-align: justify;"> How can we solve these linear systems with a quantum computer? Generally we are expressing the matrix $A$ is a linear combination of unitaries</p>

\begin{equation}
    A = \sum_{l = 1}^{L} c_l A_l
\end{equation}

<p style="text-align: justify;"> where $c_l \in \mathbb{C}$. The vector $|\mathbf{b}\rangle$ is assumed to have some unitary $B$ which prepares it from the ground state, i.e. $B|0\rangle = |\mathbf{b}\rangle$. We want to prepare a state </p>

\begin{equation}
    |\mathbf{\theta}\rangle := V(\mathbf{\theta}) |0\rangle 
\end{equation}

<p style="text-align: justify;"> which has high fidelity $| \langle \theta | x \rangle |^2$ with the solution $|x\rangle$ of the linear system $A |\mathbf{x}\rangle = |\mathbf{b}\rangle $. Equivalenlty, we want a state $|\theta\rangle$ such that $A |\theta\rangle$ is close to $|\mathbf{b}\rangle$. That is, we want to maximize the quantity

\begin{equation}
    \langle \mathbf{b} | A |\theta\rangle = \langle 0 | B^\dagger A V(\theta) |0\rangle = \sum_{l = 1}^{L} c_l \langle 0 | B^\dagger A_l V(\theta) |0\rangle .
\end{equation}

By maximizing the fidelity $|\langle \mathbf{b} | A |\theta\rangle|^2$, which involves computing individual overlap terms via Hadamard tests or Hadamard-like tests [1], one can obtain an approximate solution to the linear system. 

### <p style="text-align: center;"> The Effective Hamiltonian Approach </p>

An interesting connection to other linear systems work [4] is to note that the above prescription is equivalent to minimizing the energy of an effective Hamiltonian

\begin{equation}
    H_{A, \mathbf{b}} \equiv H := A^\dagger ( I - |\mathbf{b}\rangle \langle \mathbf{b} | ) A
\end{equation}

<p style="text-align: justify;">
This can be seen by noting that $H$ is positive semidefinite and $H |\mathbf{x}\rangle = 0$. Thus, by minimizing $\langle \theta | H |\theta \rangle$, we can get an approximate solution to the linear system. A key advantage of this approach is the ability to compute expectation values in a hardware efficient manner. (That is, not using controlled gates required by the Hadamard test but instead using classical post-processing [5-7].) We use this effective Hamiltonian approach in our simulations below.
</p>

Noting that $H = A^\dagger A - A^\dagger P_{\mathbf{b}} A$ where $P_{\mathbf{b}} := |\mathbf{b}\rangle \langle \mathbf{b} |$ is the projector onto the $|\mathbf{b}\rangle$ state, we have two terms to evaluate. For the first, one can show that

\begin{equation}
    A^\dagger A = \sum_{l = 1}^{L} |a_l|^2 I + 2 \sum_{1 = l < k}^{L - 1} \text{Re} \, [a_l^* a_k ] A_l A_k .
\end{equation}

The expectation of the first summation can be computed classically, while the second requires $L (L - 1) / 2 = )(L^2)$ circuit evaluations. 

For the second term in the Hamiltonian $H$, one can show that

\begin{equation}
    A^\dagger P_{\mathbf{b}} A = \sum_{k = 1}^{L} \sum_{l = 1}^{L} a_k^* a_l b_m A_k P_\mathbf{b} A_l .
\end{equation}

Assuming $B = \sum_{m = 1}^{M} b_m B_m$, the projector $P_\mathbf{b} = B|0\rangle \langle 0 | B^\dagger$ can be expanded as a linear combination of unitaries as well. Note that $|0\rangle\langle0| = (I + Z) / 2$. This term requires $O(L^2)$ circuit evaluations as well.

In [ ]:
def mult(a: str, b: str) -> Tuple[complex, str]:
    """Returns product a times b of two single qubit Paulis."""
    if a not in ("I", "X", "Y", "Z") or b not in ("I", "X", "Y", "Z"):
        raise ValueError("One or more invalid Pauli keys.")
        
    prod = {"XY": "Z",
            "YZ": "X",
            "ZX": "Y"}

    if a == "I":
        return (1., b)
    if b == "I":
        return (1., a)
    if a == b:
        return (1., "I")
    
    phase = 1j
    new = a + b
    if new in prod.keys():
        return (phase, prod[new])
    else:
        return (-phase, prod[b + a])
    

def multn(a: str, b: str) -> Tuple[complex, str]:
    """Returns product a times b of two n qubit Paulis."""
    if len(a) != len(b):
        raise ValueError("len(a) != len(b)")
    phase = 1
    paulis = []
    for (aterm, bterm) in zip(a, b):
        p, pauli = mult(aterm, bterm)
        phase *= p
        paulis.append(pauli)
    return phase, "".join(paulis)

In [ ]:
"""Unit tests for Pauli multiplication."""
# =================================
# Single qubit Pauli multiplication
# =================================

paulis = ("I", "X", "Y", "Z")
# Check multiplication with identity and squared terms
for p in paulis:
    assert mult("I", p) == (1., p)
    assert mult(p, "I") == (1., p)
    assert mult(p, p) == (1., "I")

# Check "cross terms"
assert mult("X", "Y") == (1j, "Z")
assert mult("Y", "X") == (-1j, "Z")
assert mult("X", "Z") == (-1j, "Y")
assert mult("Z", "X") == (1j, "Y")
assert mult("Y", "Z") == (1j, "X")
assert mult("Z", "Y") == (-1j, "X")

# ================================
# Multi-qubit Pauli multiplication
# ================================

assert multn("IXY", "ZYY") == (1j, "ZZI")
assert multn("IIIIII", "XYZXYZ") == (1., "XYZXYZ")
assert multn("YY", "ZI") == (1j, "XY")
assert multn("XZ", "ZX") == (1 + 0j, "YY")

In [ ]:
def Pb_expansion(Bcoeffs: List[complex], Bterms: List[str]) -> List[Tuple[complex, str]]:
    """Inputs B as a linear combination of unitaries and 
    outputs B|0><0|B^\dagger as a linear combination of unitaries.
    """
    if len(Bcoeffs) != len(Bterms):
        raise ValueError("len(Bcoeffs) != len(Bterms)")
    n = len(Bterms[0])
    
    # Form the |0><0| projector on n qubits
    terms = [["I", "Z"]] * n
    prod = list(product(*terms))
    Pterms = ["".join(p) for p in prod]
    
    coeffs = []
    paulis = []
    
    for ii in range(len(Bterms)):
        for jj in range(len(Pterms)):
            for kk in range(len(Bterms)):
                coeff = Bcoeffs[ii] * np.conj(Bcoeffs[kk]) / 2**n
                phase1, pauli1 = multn(Bterms[ii], Pterms[jj])
                phase2, pauli = multn(pauli1, Bterms[kk])
                
                coeffs.append(coeff * phase1 * phase2)
                paulis.append(pauli)
    return coeffs, paulis

In [ ]:
"""Unit tests for Bexpansion function."""
def test_Pbexpansion(Bcoeffs, Bterms, verbose=False):
    """Tests matrix equality for Pauli expansion of Pb = B|0><0|B^\dagger."""
    Pbcoeffs, Pbpaulis = Pb_expansion(Bcoeffs, Bterms)

    if verbose:
        print("Found Pauli expansion of Pb:")
        print(Pbcoeffs)
        print(Pbpaulis)

    # Determine the bvector
    bvec = vector(Bcoeffs, Bterms)
    if verbose:
        print("\nbvec computed from Bcoeffs and Bterms:")
        print(bvec)

    Pbmat = matrix(Pbcoeffs, Pbpaulis)
    if verbose:
        print("\nComputed Pb:")
        print(Pbmat)
        print("\nActual Pb = |b><b|:")
        print(np.outer(bvec, bvec.conj()))

    assert np.array_equal(Pbmat, np.outer(bvec, bvec.conj()))


# Single qubit with one term tests
test_Pbexpansion([1], ["I"])
test_Pbexpansion([1], ["X"])
test_Pbexpansion([1], ["Y"])
test_Pbexpansion([1], ["Z"])

# Single qubit with multiple term tests
test_Pbexpansion([1, 2], ["I", "X"])
test_Pbexpansion([1, 1, 3], ["I", "X", "Y"])

# Multi-qubit with multiple term tests
test_Pbexpansion([1, 1], ["XZ", "XZ"])
test_Pbexpansion([2, 3, 1], ["IIZ", "ZXY", "YYX"])

In [ ]:
"""Functions to attempt to minimize the number of terms in the Pauli expansion of the Hamiltonian."""
def combine_paulis(hamiltonian: List[Tuple[complex, str]]) -> List[Tuple[complex, str]]:
    """Combines identical Pauli terms in the Hamiltonian, if possible."""
    new = [list(hamiltonian[0])]
    seen_paulis = [new[0][1]]

    for (coeff, pauli) in hamiltonian[1:]:
        if pauli in seen_paulis:
            # Find the index of where the Pauli is in new
            index = seen_paulis.index(pauli)
            new[index][0] += coeff
        else:
            new.append(list((coeff, pauli)))
            seen_paulis.append(pauli)

    return new


def drop_zero(hamiltonian: List[Tuple[complex, str]]) -> List[Tuple[complex, str]]:
    """Drops Pauli terms with zero weight."""
    new = []
    for (coeff, pauli) in hamiltonian:
        if not np.isclose(abs(coeff), 0.0):
            new.append([coeff, pauli])
    return new

In [ ]:
"""Unit tests for combine Paulis."""
# Test 1
ham = [(1j, "XY"), (-1j, "XY")]
assert drop_zero(combine_paulis(ham)) == []

# Test 2
ham = [(1.0, 'II'), (1j, 'XY'), (-1j, 'XY'), (1.0, 'II')]
assert drop_zero(combine_paulis(ham)) == [[2.0, "II"]]

# Test 3
ham = [[2.0, 'II'], [-0.25, 'II'], [-0.25, 'IZ'], [-0.25, 'ZI'], 
       [-0.25, 'ZZ'], [-0.25j, 'XY'], [-0.25, 'XX'], [(0.25+0j), 'YY'], 
       [-0.25j, 'YX'], [0.25j, 'XY'], [(-0.25+0j), 'XX'], [(0.25+0j), 'YY'], 
       [0.25j, 'YX'], [-0.25, 'II'], [(0.25+0j), 'IZ'], [(0.25+0j), 'ZI'],
       [(-0.25+0j), 'ZZ']]
assert drop_zero(combine_paulis(ham)) == [[1.5, 'II'], [(-0.5+0j), 'ZZ'], [(-0.5+0j), 'XX'], [(0.5+0j), 'YY']]

In [ ]:
"""Function which inputs A, b and outputs terms of the effective Hamiltonian."""
def effective_hamiltonian(Acoeffs: List[complex], 
                          Aterms: List[str], 
                          Bcoeffs: List[complex],
                          Bterms: List[str]) -> Tuple[List[complex], List[str]]:
    """Returns the effective Hamiltonian of an input linear system Ax = B|0>.
    
    Args:
        Acoeffs: List of coefficients of A matrix.
        Aterms: Pauli strings for each coefficient of A matrix.
        Bcoeffs: List of coefficients of B matrix.
        Bterms: Pauli strings for each coefficient of B matrix.
    """
    # Input checks
    if len(Acoeffs) != len(Aterms):
        raise ValueError("len(Acoeffs) != len(Aterms)")
    if len(Bcoeffs) != len(Bterms):
        raise ValueError("len(Bceoffs) != len(Bterms)")

    if not np.isclose(np.linalg.norm(vector(Bcoeffs, Bterms)), 1.0):
        raise ValueError("The bvector must have unit norm.")

    # Initialize the Hamiltonian as a List[Tuple[complex, str]]
    ham = []
    n = len(Aterms[0])
    
    # =======================================
    # First term in Hamiltonian (A^\dagger A)
    # =======================================

    for l in range(len(Aterms)):
        for k in range(len(Aterms)):
            coeff = np.conj(Acoeffs[l]) * Acoeffs[k]
            phase, pauli = multn(Aterms[l], Aterms[k])
            ham.append(list((coeff * phase, pauli)))
    
    ham = drop_zero(combine_paulis(ham))
    
    # ============================================
    # Second term in Hamiltonian (A^\dagger P_b A)
    # ============================================
    
    # Get the Pb expansion
    Pbcoeffs, Pbterms = Pb_expansion(Bcoeffs, Bterms)
    
    for k in range(len(Aterms)):
        for l in range(len(Aterms)):
            for m in range(len(Pbterms)):
                # Compute this coefficient and term
                coeff = np.conj(Acoeffs[k]) * Acoeffs[l] * Pbcoeffs[m]
                phase1, pauli1 = multn(Aterms[k], Pbterms[m])
                phase2, pauli = multn(pauli1, Aterms[l])
                
                # Append it to the expansion
                ham.append(list((-1 * phase1 * phase2 * coeff, pauli)))
    
    return drop_zero(combine_paulis(ham))


def matrix_of_hamiltonian(ham: List[Tuple[complex, str]]) -> np.ndarray:
    """Returns a matrix representation of the Hamiltonian in the standard basis."""
    coeffs = []
    paulis = []
    for (coeff, pauli) in ham:
        coeffs.append(coeff)
        paulis.append(pauli)
    mat = matrix(coeffs, paulis)
    return mat

In [ ]:
"""Unit tests for effective hamiltonian function."""
def test_effective_hamiltonian(Acoeffs, Aterms, Amat_correct, 
                               Bcoeffs, Bterms, bvec_correct,
                               verbose = False):
    """Checks matrix equality for the effective Hamiltonian expansion.
    Ensures minimum evalue of effective Hamiltonian is 0 and
    corresponding evector is the solution to the linear system."""
    # Get the effective Hamiltonian
    ham = effective_hamiltonian(Acoeffs, Aterms, Bcoeffs, Bterms)

    # Get the matrix and vector of the linear system, and solve it
    Amat = matrix(Acoeffs, Aterms)
    assert np.array_equal(Amat, Amat_correct)
    bvec = vector(Bcoeffs, Bterms)
    if verbose:
        print("Computed bvec:")
        print(bvec)
    assert np.array_equal(bvec, bvec_correct)
    xvec = np.linalg.solve(Amat, bvec)
    xvec /= np.linalg.norm(xvec)
    
    if verbose:
        print("\nEffective Hamiltonian expansion:")
        print(ham)

    # Get the matrix of the Hamiltonian
    Heff = matrix_of_hamiltonian(ham)

    # Check correctness of the effective Hamiltonian
    Hexact = (Amat_correct.conj().T @ 
              (np.identity(len(Amat)) - np.outer(bvec_correct, bvec_correct.conj())) @ 
              Amat_correct)
    if verbose:
        print("\nExact Heff:")
        print(Hexact)
        print("\nComputed Heff:")
        print(Heff)
    assert np.allclose(Heff, Hexact)
    assert np.allclose(Heff, Heff.conj().T)
    if verbose:
        print("Heff @ xvec:")
        print(Heff @ xvec)
        print("||Heff @ xvec|| = ", np.linalg.norm(Heff @ xvec))
    assert np.isclose(np.linalg.norm(Heff @ xvec), 0.0, atol=1e-5)

    # Do the diagonalization
    evals, evecs = np.linalg.eigh(Heff)
    if verbose:
        print("Min eval:")
        print(evals[0])
        print("Evec of minimum eval:")
        print(evecs[:, 0])
        print("Solution of linear system:")
        print(xvec)
    assert np.isclose(evals[0], 0.0, atol=1e-5)
    assert np.isclose(abs(np.dot(evecs[:, 0], xvec.conj().T))**2, 1.0)

# =======================
# Tests with A = Identity
# =======================

# Test 1
Bcoeffs = [1 / 2] * 4
Bterms = ["II", "IX", "XI", "XX"]
bvec_correct = np.ones(4) / 2
Acoeffs = [1]
Aterms = ["II"]
Amat_correct = np.identity(4)
test_effective_hamiltonian(Acoeffs, Aterms, Amat_correct,
                           Bcoeffs, Bterms, bvec_correct)

# Test 2
Bcoeffs = [1 / 4] * 4
Bterms = ["II", "IZ", "ZI", "ZZ"]
bvec_correct = np.array([1, 0, 0, 0])
Acoeffs = [1]
Aterms = ["II"]
Amat_correct = np.identity(4)
test_effective_hamiltonian(Acoeffs, Aterms, Amat_correct,
                           Bcoeffs, Bterms, bvec_correct)

# Test 3
Bcoeffs = [1 / np.sqrt(2), 1 / np.sqrt(2)]
Bterms = ["II", "XY"]
bvec_correct = vector(Bcoeffs, Bterms)
Acoeffs = [1]
Aterms = ["II"]
Amat_correct = np.identity(4)
test_effective_hamiltonian(Acoeffs, Aterms, Amat_correct,
                           Bcoeffs, Bterms, bvec_correct)

# Test 4
Bcoeffs = [1 / np.sqrt(2), 1 / np.sqrt(2)]
Bterms = ["III", "XYZ"]
bvec_correct = vector(Bcoeffs, Bterms)
Acoeffs = [1]
Aterms = ["III"]
Amat_correct = np.identity(8)
test_effective_hamiltonian(Acoeffs, Aterms, Amat_correct,
                           Bcoeffs, Bterms, bvec_correct)

# ========================
# Tests with A != Identity
# ========================

# Test 5
Bcoeffs = [1 / 2] * 4
Bterms = ["II", "IX", "XI", "XX"]
bvec_correct = np.ones(4) / 2
Acoeffs = [1]
Aterms = ["IZ"]
Amat_correct = matrix(Acoeffs, Aterms)
test_effective_hamiltonian(Acoeffs, Aterms, Amat_correct,
                           Bcoeffs, Bterms, bvec_correct)

# Test 6
Bcoeffs = [1]
Bterms = ["II"]
bvec_correct = vector(Bcoeffs, Bterms)
Acoeffs = [1, 1]
Aterms = ["IZ", "XX"]
Amat_correct = matrix(Acoeffs, Aterms)
test_effective_hamiltonian(Acoeffs, Aterms, Amat_correct,
                           Bcoeffs, Bterms, bvec_correct)

# Test 7
Bcoeffs = [1]
Bterms = ["ZY"]
bvec_correct = vector(Bcoeffs, Bterms)
Acoeffs = [1, 1]
Aterms = ["IZ", "XX"]
Amat_correct = matrix(Acoeffs, Aterms)
test_effective_hamiltonian(Acoeffs, Aterms, Amat_correct,
                           Bcoeffs, Bterms, bvec_correct)

# ===========================
# Test example linear systems
# ===========================

# Test 8: Three qubit example from VQLS paper
Bcoeffs = [1 / 2**(3/2)] * 8
Bterms = ["XXX", "XXZ", "XZX", "XZZ", "ZXX", "ZXZ", "ZZX", "ZZZ"]
bvec_correct = vector(Bcoeffs, Bterms)
Acoeffs = [1, 0.2, 0.2]
Aterms = ["III", "XII", "XZI"]
Amat_correct = matrix(Acoeffs, Aterms)
test_effective_hamiltonian(Acoeffs, Aterms, Amat_correct,
                           Bcoeffs, Bterms, bvec_correct)

# Test 9: Five qubit example from VQLS paper
Bcoeffs = [1 / 2**(5/2)] * 32
paulis = [["X", "Z"]] * 5
prods = list(product(*paulis))
Bterms = ["".join(p) for p in prods]
bvec_correct = vector(Bcoeffs, Bterms)
Acoeffs = [1, 0.2, 0.2]
Aterms = ["IIIII", "XIIII", "XZIII"]
Amat_correct = matrix(Acoeffs, Aterms)
test_effective_hamiltonian(Acoeffs, Aterms, Amat_correct,
                           Bcoeffs, Bterms, bvec_correct)

# <p style="text-align: center;"> Linear System 1 </p>

Below we examine the number of terms in the effective Hamiltonian for the first example linear system.

In [ ]:
"""Terms in effective Hamiltonian for first linear system."""
n = 5
Bcoeffs = [1 / 2**(n / 2)] * 2**n
paulis = [["X", "Z"]] * n
prods = list(product(*paulis))
Bterms = ["".join(p) for p in prods]
Acoeffs = [1, 0.2, 0.2]
Aterms = ["IIIII", "XIIII", "XZIII"]

ham = effective_hamiltonian(Acoeffs, Aterms, Bcoeffs, Bterms)

print("Number of terms:", len(ham))
print(*ham, sep="\n")

#### TODO: Utilize simultaneous measurements

We can compute the expectation of this effective Hamiltonian 

In [ ]:
"""Parameters/constants."""
# Computer to run on
qcomputer = "Aspen-7-5Q-B"  # Five qubit lattice for first linear system
# qcomputer = "Aspen-7-3Q-B"  # Three qubit lattice for testing optimizer
lattice5q = get_qc(qcomputer, as_qvm=True)  # Change to as_qvm=False to run on QC. Must have reservation.

In [ ]:
"""Visualize the qubit connectivity."""
graph = lattice5q.qubit_topology()
nx.draw(graph)

## <p style="text-align: center;"> Variational Ansatz </p>

In [ ]:
"""Define an ansatz circuit."""
def yansatz(computer):
    """Returns a circuit with a product state ansatz."""
    n = len(computer.qubits())
    # Get a circuit and classical memory register
    circ = Program()
    creg = circ.declare("ro", memory_type="BIT", memory_size=n)

    # Define parameters for the ansatz
    angles = circ.declare("theta", memory_type="REAL", memory_size=n)

    # Add the ansatz
    circ += [gates.RY(angles[ii], computer.qubits()[ii]) for ii in range(n)]
    
    return circ, creg

In [ ]:
def yansatzCZ(computer):
    """Returns a Ry ansatz with some entanglement."""
    # Grab the qubits
    qubits = tuple(computer.qubits())
    n = len(qubits)
    
    # Get a circuit and classical memory register
    circ = Program()
    creg = circ.declare("ro", memory_type="BIT", memory_size=n)

    # Define parameters for the ansatz
    angles = circ.declare("theta", memory_type="REAL", memory_size=n)

    # Add the ansatz
    circ += [gates.RY(angles[ii], qubits[ii]) for ii in range(n)]
    circ += [gates.CZ(qubits[ii], qubits[ii + 1]) for ii in range(n - 1)]
    
    return circ, creg

In [1]:
"""Get an ansatz."""
circ, creg = yansatzCZ(lattice5q)
print(circ)

NameError: name 'yansatzCZ' is not defined

In [ ]:
def expectation(angles: List[float], coeff: complex, pauli: str,
                ansatz, computer, shots: int = 10000,
                verbose: bool = False) -> float:
    """Returns coeff * <\theta| paulii |\theta>.
    
    Args:
        angles
        coeff:
        pauli:
        ansatz:
        computer:
        shots:
        verbose: 
    """
    if np.isclose(np.imag(coeff), 0.0):
        coeff = np.real(coeff)

    if set(pauli) == {"I"}:
        return coeff
    
    # Set up the circuit
    circuit = ansatz.copy()
    qubits = computer.qubits()
    measured = []
    for (q, p) in enumerate(pauli):
        if p == "X":
            circuit += [gates.H(qubits[q]), gates.MEASURE(qubits[q], creg[q])]
            measured.append(qubits[q])
        elif p == "Y":
            circuit += [gates.S(qubits[q]), gates.H(qubits[q]), gates.MEASURE(qubits[q], creg[q])]
            measured.append(qubits[q])
        elif p == "Z":
            circuit += [gates.MEASURE(qubits[q], creg[q])]
            measured.append(qubits[q])
    
    # Run the circuit
    circuit.wrap_in_numshots_loop(shots)
    
    if verbose:
        print(f"Computing {coeff} x <theta|{pauli}|theta>...")
        print("\nCircuit to be executed:")
        print(circuit)
    
    # Execute the circuit
    executable = computer.compile(circuit)
    res = computer.run(executable, memory_map={"theta": angles})
    
    if verbose:
        print("\nResults:")
        print(f"{len(res)} total measured bit strings.")
        print(res)
    
    # Do the postprocessing
    tot = 0.0
    for vals in res:
        tot += (-1)**sum(vals)
    return coeff * tot / shots

In [ ]:
"""Unit tests for expectation."""
expectation([0] * n, 1, "ZZZII", circ, lattice5q, shots=10000, verbose=True)

In [ ]:
def energy(angles, hamiltonian, ansatz, computer, shots=10000, verbose=False):
    """Returns <theta|H|theta>.
    
    Args:
        angles:
        ham:
        ansatz:
        computer:
        shots:
        verbose:
    """
    value = 0.0
    for (coeff, pauli) in hamiltonian:
        value += expectation(angles, coeff, pauli, ansatz, computer, shots, verbose)
    return value

In [ ]:
"""Unit tests for energy."""
energy([0] * n, ham, circ, lattice5q, shots=10000, verbose=False)

In [ ]:
"""Cost function for linear system 1."""
def costLS1(angles):
    val = energy(angles, ham, circ, lattice5q, shots=10000)
    print("Current angles:", angles)
    print("Current energy:", val)
    return val

In [ ]:
"""Do the optimization."""
start = time.time()
res = minimize(costLS1, x0=np.random.randn(n), method="COBYLA")
print("Runtime:", (time.time() - start) // 60, "minutes.")

### <p style="text-align: center;"> Compare the Quantum and Classical Solutions </p>

In [ ]:
print("Optimal angles:", res.x)

In [ ]:
"""Define function for taking (circuit, optimal angles) --> wavefunction."""
def qsolution(ansatz, opt_angles):
    """Returns the wavefunction of the ansatz at the optimal angles."""
    prog = Program()
    memory_map = {"theta": opt_angles}
    for name, arr in memory_map.items():
        for index, value in enumerate(arr):
            prog += gates.MOVE(gates.MemoryReference(name, offset=index), value)

    ansatz = prog + ansatz
    soln = pyquil.quil.percolate_declares(ansatz)
    
    wfsim = pyquil.api.WavefunctionSimulator()
    return wfsim.wavefunction(soln).amplitudes

In [ ]:
qxvec = qsolution(circ, res.x)
print(qxvec)

In [ ]:
"""Compare to classical solution."""
# Get the classical solution
Amat = matrix(Acoeffs, Aterms)
bvec = vector(Bcoeffs, Bterms)
xvec = np.linalg.solve(Amat, bvec)
xvec /= np.linalg.norm(xvec)

# Compute the fidelity
fidelity = abs(np.dot(qxvec.conj(), xvec))**2
print(r"| < xquantum | xclassical > |^2 =", round(fidelity, 2))

**SAVE THIS CELL!**

In [ ]:
"""SAVE THIS CELL!

Optimal angles for Ry product ansatz. Energy of effective Hamiltonian is near 0.001 here.
"""
LS1opt_angles = [1.60380032, 1.88421818, 1.61131353, 1.51679728, 1.44425814]

# <p style="text-align: center;"> Linear System 2 </p>

The second linear system we consider is formed from the Ising model.

\begin{equation}
    A_2 := \frac{1}{\zeta} \left(\eta I + \sum_{j=1}^{n} X_j + J \sum_{j=1}^{n-1}Z_jZ_{j+1} \right)
\end{equation}

In [ ]:
def get_Acoeffs_Aterms_LS2(n: int, zeta: float, eta: float, J: float) -> List[Tuple[complex, str]]:
    """Returns linear system 2 expressed as weighted Pauli sum."""
    if n < 0:
        raise ValueError("Number of qubits must be >= 1.")
    # Add the identity
    Acoeffs = [eta / zeta]
    Aterms = ["I" * n]
    
    # Add the X terms
    Acoeffs += [1 / zeta] * n
    xbase = "X" + "I" * (n - 1)
    for ii in range(n, 0, -1):
        Aterms.append(xbase[ii:] + xbase[:ii])
        
    # Add the ZZ terms
    Acoeffs += [J / zeta] * (n - 1)
    zzbase = "ZZ" + "I" * (n - 2)
    for ii in range(n, 1, -1):
        Aterms.append(zzbase[ii:] + zzbase[:ii])
    
    return Acoeffs, Aterms

In [ ]:
"""Unit tests for getting LS2."""
Acoeffs, Aterms = get_Acoeffs_Aterms_LS2(4, 1, 1, 0.1)
assert Acoeffs == [1.0, 1.0, 1.0, 1.0, 1.0, 0.1, 0.1, 0.1]
assert Aterms == ['IIII', 'XIII', 'IXII', 'IIXI', 'IIIX', 'ZZII', 'IZZI', 'IIZZ']

In [ ]:
"""Terms in effective Hamiltonian for second linear system."""
# Number of qubits
n = 5

# Constants
zeta = 1.0
J = 0.1
eta = 1.0

# |b> = H|0>
Bcoeffs = [1 / 2**(n / 2)] * 2**n
paulis = [["X", "Z"]] * n
prods = list(product(*paulis))
Bterms = ["".join(p) for p in prods]

# Bcoeffs = [1 / 2**(1 / 2)] * 2
# Bterms = ["IXIII", "XIXII"]

# Linear system in weighted Paulis
Acoeffs, Aterms = get_Acoeffs_Aterms_LS2(n, zeta, eta, J)

ham2 = effective_hamiltonian(Acoeffs, Aterms, Bcoeffs, Bterms)

print("Number of terms:", len(ham2))
print(*ham2, sep="\n")

In [ ]:
"""View matrix of linear system we are solving."""
print(matrix_of_hamiltonian(ham2))

In [ ]:
"""Parameters/constants."""
# Computer to run on
qcomputer = "Aspen-7-5Q-B"  # Five qubit lattice for first linear system
# qcomputer = "Aspen-7-3Q-B"  # Three qubit lattice for testing optimizer
lattice3q = get_qc(qcomputer, as_qvm=True)  # Change to as_qvm=False to run on QC. Must have reservation.

In [ ]:
"""Get an ansatz."""
circ, creg = yansatz(lattice3q)
print(circ)

In [ ]:
"""Cost function for linear system 2."""
def costLS2(angles):
    val = energy(angles, ham2, circ, lattice3q, shots=10000)
    print("Current angles:", angles)
    print("Current energy:", val)
    return val

In [ ]:
"""Do the optimization."""
start = time.time()
res = minimize(costLS2, x0=np.random.randn(n), method="COBYLA")
print("Runtime:", (time.time() - start) // 60, "minutes.")

In [ ]:
print("Optimal angles:", res.x)

In [ ]:
qxvec = qsolution(circ, res.x)
print(qxvec)

In [ ]:
"""Compare to classical solution."""
# Get the classical solution
Amat = matrix(Acoeffs, Aterms)
bvec = vector(Bcoeffs, Bterms)
xvec = np.linalg.solve(Amat, bvec)
xvec /= np.linalg.norm(xvec)

# Compute the fidelity
fidelity = abs(np.dot(qxvec.conj(), xvec))**2
print(r"| < xquantum | xclassical > |^2 =", round(fidelity, 2))

In [ ]:
xvec

## Running on a Quantum Computer

This notebook was made into a script that could be run on Rigetti's Quantum Cloud Services. In particular, we used a three qubit lattice "Aspen-4-3Q-A," the fidelity/noise characterization of which can be found online. In the cell below, we load energy vs iteration data that was computed using our VQE algorithm on this lattice. We then plot it against the simulator result to compare the difference.

In [ ]:
"""Plot the QPU energy vs iteration data obtained by running on Rigetti QCS."""
# Read in the files
qpu_energy1 = np.loadtxt("qpu-energy-iteration1.txt")
qpu_energy2 = np.loadtxt("qpu-energy-iteration2.txt")

# Do the plotting
plt.figure(figsize=(10, 5))
plt.plot(energies, "--o", linewidth=3, label="Simulator")
plt.xlabel("Iteration", fontsize=14, fontweight="bold")
plt.ylabel("Energy", fontsize=14, fontweight="bold")

plt.plot(qpu_energy1, "--o", linewidth=3, label="Raw QPU Run 1")
plt.plot(qpu_energy2, "--o", linewidth=3, label="Raw QPU Run 2")

# Put a line at the actual ground state energy (see below)
GSENERGY = 0.53232723
plt.plot(GSENERGY * np.ones_like(energies), linewidth=3, label="Analytic Energy")

plt.grid()
plt.legend();

## Error Mitigation

The QPU data above has a significant vertical shift from the simulator data, but generally the same "shape." This shift is present because of decoherence, gate application errors, measurement errors, and other noise in the system. It can be accounted for by running a set of "benchmark circuits" on the QPU prior to running the actual algorithm. These benchmark circuits are simple circuits, such as NOT and MEASURE, for which one knows the actual output. The vertical shift in this benchmark circuit can then be tested for and subtracted from the final computed energies to get more accurate results. Other methods are also possible, and this is an area of active research.

Here, we employ a similar idea, but instead of running a benchmark circuit, we just subtract off the difference of the initial energies on the QPU and on the simulator. The cell below implements this and plots the result again.

In [ ]:
"""Plot the error mitigated QPU energy vs iteration data obtained by running on Rigetti QCS."""
# Constant shift amount. In practice, this would be obtained by running a "benchmark circuit."
# Here, we just set the value based on the above curves
shift = 0.3

# Read in the files
qpu_energy1 = np.loadtxt("qpu-energy-iteration1.txt")
qpu_energy2 = np.loadtxt("qpu-energy-iteration2.txt")

# Subtract off the shift
qpu_energy1 -= shift
qpu_energy2 -= shift

# Do the plotting
plt.figure(figsize=(10, 5))
plt.plot(energies, "--o", linewidth=3, label="Simulator")
plt.xlabel("Iteration", fontsize=14, fontweight="bold")
plt.ylabel("Energy", fontsize=14, fontweight="bold")

plt.plot(qpu_energy1, "--o", linewidth=3, label="Error Mitigated QPU Run 1")
plt.plot(qpu_energy2, "--o", linewidth=3, label="Error Mitigated QPU Run 2")

# Put a line at the actual ground state energy (see below)
GSENERGY = 0.53232723
plt.plot(GSENERGY * np.ones_like(energies), linewidth=3, label="Analytic Energy")

plt.grid()
plt.legend();

## Conclusions

## Acknowledgements

Thank coauthors and Rigetti.

## References

[1] VQLS

[2] HHL

[3] QLSP via QSVE

[4] Yigit's adiabatic algorithm

[5] VQE Original paper

[6] Theory of VQE

[7] Ryan LaRose et al., [Implementing variational quantum algorithms](https://github.com/rmlarose/qcbq/blob/master/tutorials/04qaoa/qcbq_tutorial4b_implement_qaoa-INSTRUCTOR.ipynb), MSU-IBM Quantum computing bootcamp with Qiskit, 2019. 